In [ ]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# import model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

### Load Data

In [ ]:
# load data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Data Preprocessing

In [ ]:
# cat column
cat_col = train.select_dtypes(include=['object']).columns

# NA column
na_col = train.columns[train.isna().sum() > 0]

# NA column with meaning (from the description)
na_col_meaning = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]

# NA column meaning - all NA col
na_col_no_meaning = [col for col in na_col if col not in na_col_meaning]

In [11]:
x_train, x_val, y_train, y_val = train_test_split(train.drop(columns=['SalePrice']), train['SalePrice'], test_size=0.2, random_state=42)

In [13]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

ValueError: could not convert string to float: 'RL'

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
